In [12]:
import numpy as np
import pandas as pd

from time import sleep

import pickle

from pprint import pprint

from hab.airtable.airtable import AirtableHAB, airtable_oauth
from hab.serpapi import serpapi, locations
from hab.clean_functions import clean

In [2]:
df_locations = locations.country_locations("es")

In [3]:
df_locations.shape

(546, 9)

In [4]:
df_locations

,id,google_id,google_parent_id,name,canonical_name,country_code,target_type,reach,gps
0,585069bfee19ad271e9bc658,2724,NaN,Spain,Spain,ES,Country,33900000.0,"[-3.74922, 40.46366700000001]"
1,585069bdee19ad271e9bc183,20282,2724.0,Community of Madrid,"Community of Madrid,Spain",ES,Autonomous Community,11600000.0,"[-3.7038322, 40.4167515]"
2,585069bdee19ad271e9bc17e,20278,2724.0,Catalonia,"Catalonia,Spain",ES,Autonomous Community,11200000.0,"[1.5208624, 41.5911589]"
3,585069a6ee19ad271e9b5cda,1005493,20282.0,Madrid,"Madrid,Community of Madrid,Spain",ES,City,11000000.0,"[-3.7038322, 40.4167515]"
4,585069bdee19ad271e9bc176,20270,20278.0,Barcelona,"Province of Barcelona,Catalonia,Spain",ES,Province,10100000.0,"[2.1734035, 41.3850639]"
...,...,...,...,...,...,...,...,...,...
541,585069e2ee19ad271e9c6396,9047050,20290.0,Cantabria,"Province of Cantabria,Cantabria,Spain",ES,Province,NaN,"[-3.9878427, 43.1828396]"
542,585069e2ee19ad271e9c6397,9047051,20285.0,Navarre,"Province of Navarre,Navarre,Spain",ES,Province,NaN,"[-1.6760691, 42.6953909]"
543,585069e2ee19ad271e9c6399,9047053,20281.0,La Rioja,"Province of La Rioja,La Rioja,Spain",ES,Province,NaN,"[-2.539603, 42.2870733]"
544,585069e2ee19ad271e9c639c,9047056,20284.0,Murcia,"Province of Murcia,Murcia,Spain",ES,Province,NaN,"[-1.1306544, 37.9922399]"


In [20]:
%%time

df = pd.DataFrame()

for pagination in range(21):
    
    q_params = {"q"             : "data analyst",
                "api_key"       : "1853c675785a4bed28988290f5aad8bd48bb32ba313cc5673d16f671f9a5b226",
                "location"      : "Spain",
                "start"         : pagination*10,
                "gl"            : "es",
                "hl"            : "es",
                "google_domain" : "google.es"}

    response = serpapi.job_search(**q_params)

    df_response = pd.json_normalize(response["jobs_results"])

    df_response.columns = [x.split(".")[0] if len(x.split(".")) == 1 else x.split(".")[-1] for x in df_response.columns]

#     df_response = df_response[["job_id", "title", "company_name", "location", "source", "description", "country", "date_posted", "contract_type", "remote_work"]]

    df = pd.concat(objs = [df, df_response], ignore_index = True)
    
df = df.drop_duplicates(subset = "job_id").reset_index(drop = True)

df.to_csv(f"prueba_{'data science'.replace(' ', '_')}_10pages_es_20230429.csv", index = False)
df.to_pickle(f"prueba_{'data science'.replace(' ', '_')}_10pages_es_20230429.pkl")
    
# Clean "via" columns
df["via"] = df["via"].str.replace("via ", "")
df["via"] = df["via"].str.replace("a través de ", "")

# Clean "location" column
df["location"] = df["location"].apply(lambda x : clean.clean_regular_brackets(x))

# Creates "country" column
df["country"] = df["location"].apply(lambda x : x.split()[-1] if (x != "Anywhere") and ("," in x) else x)

# Removes "country" from "location"
df["location"] = df["location"].apply(lambda x : " ".join(x.split()[:-1]).strip(",") if len(x.split()) > 1 else x)


df.rename(mapper  = {"via"            : "source",
                     "posted_at"      : "date_posted",
                     "schedule_type"  : "contract_type",
                     "work_from_home" : "remote_work"},
          axis    = 1,
          inplace = True)

https://serpapi.com/search


KeyboardInterrupt: 

In [7]:
df

,title,company_name,location,source,description,job_highlights,related_links,thumbnail,extensions,job_id,date_posted,contract_type,country
0,Data Analyst,Siemens Healthineers,Tres,LinkedIn,Siemens Mobility is a separately managed compa...,[{'items': ['Siemens Mobility is a separately ...,[{'link': 'http://www.siemens-healthineers.com...,https://encrypted-tbn0.gstatic.com/images?q=tb...,"[hace 20 horas, A tiempo completo]",eyJqb2JfdGl0bGUiOiJEYXRhIEFuYWx5c3QiLCJodGlkb2...,hace 20 horas,A tiempo completo,Cantos
1,Senior Data Analyst & Team Leader,CHEP,Madrid,LinkedIn,⚠️⚠️ Please read before applying ⚠️⚠️\n• This ...,[{'items': ['⚠️⚠️ Please read before applying ...,"[{'link': 'http://www.chep.com/', 'text': 'che...",https://encrypted-tbn0.gstatic.com/images?q=tb...,"[hace 1 día, A tiempo completo]",eyJqb2JfdGl0bGUiOiJTZW5pb3IgRGF0YSBBbmFseXN0IF...,hace 1 día,A tiempo completo,Madrid
2,Data Analyst,Vermut,España,LinkedIn,About Vermut\n\nVermut is a community marketpl...,[{'items': ['About Vermut Vermut is a communi...,[{'link': 'https://www.google.es/search?gl=es&...,https://encrypted-tbn0.gstatic.com/images?q=tb...,"[hace 1 día, A tiempo completo]",eyJqb2JfdGl0bGUiOiJEYXRhIEFuYWx5c3QiLCJodGlkb2...,hace 1 día,A tiempo completo,España
3,Freelance Data Analyst (Python),HAYS,Madrid,Tecnoempleo,En HAYS estamos colaborando con una empresa qu...,[{'items': ['En HAYS estamos colaborando con u...,"[{'link': 'http://www.hays.com/', 'text': 'hay...",https://encrypted-tbn0.gstatic.com/images?q=tb...,"[hace 5 días, A tiempo completo]",eyJqb2JfdGl0bGUiOiJGcmVlbGFuY2UgRGF0YSBBbmFseX...,hace 5 días,A tiempo completo,Madrid
4,Data Analyst,PSS,Madrid,Trabajo.org,Quieres formar parte de nuestro equipo Si nece...,[{'items': ['Quieres formar parte de nuestro e...,[{'link': 'https://www.google.es/search?gl=es&...,https://encrypted-tbn0.gstatic.com/images?q=tb...,"[hace 12 horas, A tiempo completo]",eyJqb2JfdGl0bGUiOiJEYXRhIEFuYWx5c3QiLCJodGlkb2...,hace 12 horas,A tiempo completo,Madrid
...,...,...,...,...,...,...,...,...,...,...,...,...,...
199,Regulatory Data Analyst,BME | Bolsas y Mercados Españoles,Madrid,LinkedIn,Job description:\n\nBME is the infrastructure ...,[{'items': ['Job description: BME is the infr...,"[{'link': 'http://www.bolsasymercados.es/', 't...",https://encrypted-tbn0.gstatic.com/images?q=tb...,"[hace 20 días, A tiempo completo]",eyJqb2JfdGl0bGUiOiJSZWd1bGF0b3J5IERhdGEgQW5hbH...,hace 20 días,A tiempo completo,Madrid
200,Corporate Data Analyst,Leroy Merlin - España,Alcobendas,LinkedIn,Centro de Servicios a Tienda y Desarrollo de N...,[{'items': ['Centro de Servicios a Tienda y De...,"[{'link': 'http://www.leroymerlin.es/', 'text'...",https://encrypted-tbn0.gstatic.com/images?q=tb...,[A tiempo completo],eyJqb2JfdGl0bGUiOiJDb3Jwb3JhdGUgRGF0YSBBbmFseX...,NaN,A tiempo completo,Alcobendas
201,Master Data Analyst,SGS,Madrid,LinkedIn,Company Description\n\nWe are SGS – the world’...,[{'items': ['Company Description We are SGS –...,"[{'link': 'http://www.sgs.com/', 'text': 'sgs....",https://encrypted-tbn0.gstatic.com/images?q=tb...,"[hace 15 días, A tiempo completo]",eyJqb2JfdGl0bGUiOiJNYXN0ZXIgRGF0YSBBbmFseXN0Ii...,hace 15 días,A tiempo completo,Madrid
202,Prácticas Data Analyst,Fintonic,Madrid,LinkedIn,Fintonic es una start-up en el sector fintech ...,[{'items': ['Fintonic es una start-up en el se...,"[{'link': 'http://www.fintonic.com/', 'text': ...",https://encrypted-tbn0.gstatic.com/images?q=tb...,"[hace 11 días, A tiempo completo]",eyJqb2JfdGl0bGUiOiJQcsOhY3RpY2FzIERhdGEgQW5hbH...,hace 11 días,A tiempo completo,Madrid


In [24]:
df1 = pd.read_pickle(filepath_or_buffer = "prueba_data_science_10pages_es_20230429.pkl")

df1

,title,company_name,location,via,description,job_highlights,related_links,thumbnail,extensions,job_id,posted_at,schedule_type
0,Data Analyst,Siemens Healthineers,Tres Cantos,a través de LinkedIn,Siemens Mobility is a separately managed compa...,[{'items': ['Siemens Mobility is a separately ...,[{'link': 'http://www.siemens-healthineers.com...,https://encrypted-tbn0.gstatic.com/images?q=tb...,"[hace 20 horas, A tiempo completo]",eyJqb2JfdGl0bGUiOiJEYXRhIEFuYWx5c3QiLCJodGlkb2...,hace 20 horas,A tiempo completo
1,Senior Data Analyst & Team Leader,CHEP,Madrid,a través de LinkedIn,⚠️⚠️ Please read before applying ⚠️⚠️\n• This ...,[{'items': ['⚠️⚠️ Please read before applying ...,"[{'link': 'http://www.chep.com/', 'text': 'che...",https://encrypted-tbn0.gstatic.com/images?q=tb...,"[hace 1 día, A tiempo completo]",eyJqb2JfdGl0bGUiOiJTZW5pb3IgRGF0YSBBbmFseXN0IF...,hace 1 día,A tiempo completo
2,Data Analyst,Vermut,España,a través de LinkedIn,About Vermut\n\nVermut is a community marketpl...,[{'items': ['About Vermut Vermut is a communi...,[{'link': 'https://www.google.es/search?gl=es&...,https://encrypted-tbn0.gstatic.com/images?q=tb...,"[hace 1 día, A tiempo completo]",eyJqb2JfdGl0bGUiOiJEYXRhIEFuYWx5c3QiLCJodGlkb2...,hace 1 día,A tiempo completo
3,Freelance Data Analyst (Python),HAYS,Madrid,a través de Tecnoempleo,En HAYS estamos colaborando con una empresa qu...,[{'items': ['En HAYS estamos colaborando con u...,"[{'link': 'http://www.hays.com/', 'text': 'hay...",https://encrypted-tbn0.gstatic.com/images?q=tb...,"[hace 5 días, A tiempo completo]",eyJqb2JfdGl0bGUiOiJGcmVlbGFuY2UgRGF0YSBBbmFseX...,hace 5 días,A tiempo completo
4,Data Analyst,PSS,Madrid,a través de Trabajo.org,Quieres formar parte de nuestro equipo Si nece...,[{'items': ['Quieres formar parte de nuestro e...,[{'link': 'https://www.google.es/search?gl=es&...,https://encrypted-tbn0.gstatic.com/images?q=tb...,"[hace 12 horas, A tiempo completo]",eyJqb2JfdGl0bGUiOiJEYXRhIEFuYWx5c3QiLCJodGlkb2...,hace 12 horas,A tiempo completo
...,...,...,...,...,...,...,...,...,...,...,...,...
199,Regulatory Data Analyst,BME | Bolsas y Mercados Españoles,Madrid,a través de LinkedIn,Job description:\n\nBME is the infrastructure ...,[{'items': ['Job description: BME is the infr...,"[{'link': 'http://www.bolsasymercados.es/', 't...",https://encrypted-tbn0.gstatic.com/images?q=tb...,"[hace 20 días, A tiempo completo]",eyJqb2JfdGl0bGUiOiJSZWd1bGF0b3J5IERhdGEgQW5hbH...,hace 20 días,A tiempo completo
200,Corporate Data Analyst,Leroy Merlin - España,Alcobendas,a través de LinkedIn,Centro de Servicios a Tienda y Desarrollo de N...,[{'items': ['Centro de Servicios a Tienda y De...,"[{'link': 'http://www.leroymerlin.es/', 'text'...",https://encrypted-tbn0.gstatic.com/images?q=tb...,[A tiempo completo],eyJqb2JfdGl0bGUiOiJDb3Jwb3JhdGUgRGF0YSBBbmFseX...,NaN,A tiempo completo
201,Master Data Analyst,SGS,Madrid,a través de LinkedIn,Company Description\n\nWe are SGS – the world’...,[{'items': ['Company Description We are SGS –...,"[{'link': 'http://www.sgs.com/', 'text': 'sgs....",https://encrypted-tbn0.gstatic.com/images?q=tb...,"[hace 15 días, A tiempo completo]",eyJqb2JfdGl0bGUiOiJNYXN0ZXIgRGF0YSBBbmFseXN0Ii...,hace 15 días,A tiempo completo
202,Prácticas Data Analyst,Fintonic,Madrid,a través de LinkedIn,Fintonic es una start-up en el sector fintech ...,[{'items': ['Fintonic es una start-up en el se...,"[{'link': 'http://www.fintonic.com/', 'text': ...",https://encrypted-tbn0.gstatic.com/images?q=tb...,"[hace 11 días, A tiempo completo]",eyJqb2JfdGl0bGUiOiJQcsOhY3RpY2FzIERhdGEgQW5hbH...,hace 11 días,A tiempo completo


In [19]:
df1["location"].apply(lambda x : clean.clean_regular_brackets(x)).value_counts()

Madrid                        108
España                         34
Valencia                       15
Zaragoza                       10
Gijón                           6
Alcobendas                      5
Pozuelo de Alarcón              5
Boadilla del Monte              4
Tres Cantos                     2
Bilbao                          2
Sarriguren                      2
Sevilla                         2
Alicante                        2
Liria                           1
Murcia                          1
Leioa                           1
Cantabria                       1
Cuenca                          1
Escúzar                         1
San Sebastián de los Reyes      1
Name: location, dtype: int64

In [38]:
"hola hola".capitalize()

'Hola hola'

In [14]:
df1["schedule_type"].value_counts()

A tiempo completo    196
Prácticas              5
Contratista            2
A tiempo parcial       1
Name: schedule_type, dtype: int64

In [8]:
map_contract_type = {"Tiempo completo" : "Full-time",
                     "Pasantía"        : "Internship",
                     "Medio tiempo"    : "Part-time",
                     "Contratista"     : "Contractor"}

df["contract_type"] = df["contract_type"].map(map_contract_type)

In [9]:
# for k, v in df["job_highlights"][0][0].items():
#     print(k)
#     print(v)
    
#     print("*"*30)

In [10]:
df = df[["job_id", "title", "company_name", "location", "source", "description", "country", "date_posted", "contract_type"]]

In [11]:
df

,job_id,title,company_name,location,source,description,country,date_posted,contract_type
0,eyJqb2JfdGl0bGUiOiJTZW5pb3IgU2NpZW50aXN0LCBDb2...,"Senior Scientist, Computational Oncology, Onco...",10001248 - Informatics Specialist,Barcelona,AstraZeneca Careers,The Computational Oncology group drives high-i...,España,hace 4 días,Full-time
1,eyJqb2JfdGl0bGUiOiJEYXRhIEdvdmVybmFuY2UgU3BlY2...,"Data Governance Specialist, Oncology Data Scie...",10001225 - Data Governance & Policy Specialist,Barcelona,AstraZeneca Careers,"About AstraZeneca\n\nAstraZeneca is a global, ...",España,hace 5 días,Full-time
2,eyJqb2JfdGl0bGUiOiJTZW5pb3IgRGF0YSBTY2llbnRpc3...,"Senior Data Scientist (They, She, He)",Xcelirate - We’re Hiring!,España,LinkedIn,Description\n\nAbout Xcelirate\n\nWe like to t...,España,hace 4 días,Full-time
3,eyJqb2JfdGl0bGUiOiJEYXRhIFNjaWVudGlzdCAoUHJpY2...,Data Scientist (Pricing Actuary),Zurich Insurance Company Ltd.,Madrid,Zurich Careers,"#LI-Hybrid\n\n¿Qué puede salir bien, cuando pu...",España,hace 5 días,Full-time
4,eyJqb2JfdGl0bGUiOiJEYXRhIFNjaWVuY2UgTWFzdGVyIE...,Data Science Master Inventory Management,HP,España,The Muse,Data Science Master for Inventory Management\n...,España,hace 1 día,Full-time
...,...,...,...,...,...,...,...,...,...
200,eyJqb2JfdGl0bGUiOiJGb3J3YXJkLWRlcGxveWVkIERhdG...,Forward-deployed Data Engineer,"BCG GAMMA, Boston Consulting Group",Madrid,Boston Consulting Group,Who We Are\n\nBoston Consulting Group partners...,España,NaN,Full-time
201,eyJqb2JfdGl0bGUiOiJGb3J3YXJkLWRlcGxveWVkIExlYW...,Forward-deployed Lead Data Engineer,"BCG GAMMA, Boston Consulting Group",Madrid,Boston Consulting Group,Who We Are\n\nBoston Consulting Group partners...,España,NaN,Full-time
202,eyJqb2JfdGl0bGUiOiJGb3J3YXJkLWRlcGxveWVkIExlYW...,Forward-deployed Lead Data Engineer,"BCG GAMMA, Boston Consulting Group",Madrid,Boston Consulting Group,Who We Are\n\nBoston Consulting Group partners...,España,NaN,Full-time
203,eyJqb2JfdGl0bGUiOiJGb3J3YXJkLWRlcGxveWVkIExlYW...,Forward-deployed Lead Data Engineer,"BCG GAMMA, Boston Consulting Group",Madrid,Boston Consulting Group,Who We Are\n\nBoston Consulting Group partners...,España,NaN,Full-time


In [12]:
hab = AirtableHAB(**airtable_oauth)

In [13]:
df = hab.fill_na(df)

In [15]:
hab.airtable_post(df)

205 added to HACK A JOB Project table.


In [22]:
import re

def extract_salary(string):
    pattern = r"\$?\d{1,3}(?:,?\d{3})*(?:\.\d{2})?"
    matches = re.findall(pattern, string)
    if matches:
        return matches
    else:
        return None

# Example usage
salary_string = "The salary for this position is $50,000 per year."
salary = extract_salary(salary_string)
if salary:
    print("Extracted salary:", salary)
else:
    print("No salary found.")

Extracted salary: $50,000


In [26]:
df1["description"].apply(lambda x : extract_salary(x))

0         160
1           3
2         202
3           3
4           2
        ...  
199         2
200         5
201    97,000
202      None
203       100
Name: description, Length: 204, dtype: object

In [28]:
df1["description"][201]

'Company Description\n\nWe are SGS – the world’s leading testing, inspection and certification company. We are recognized as the global benchmark for sustainability, quality and integrity. Our 97,000 employees operate a network of 2,650 offices and laboratories, working together to enable a better, safer and more interconnected world.\n\nJob Description\n\nWorking on our Digital Labs project, you will play a key role in the configuration of our new Laboratory Information Management Systems (LIMS).\n\nOur Digital Labs initiative is a flagship IT project and you will contribute to the number one leader in the TIC industry having a best-in-class IT infrastructure.\n\nAs Master Data Anlyst, you will join our Rollout Team and your main responsibilities will be:\n• Setup and maintenance on SGS LIMS solutions which benefit and support the Digital Laboratory Program.\n• Communicate with global and regional technical staff to help drive LIMS solutions supporting the Digital Laboratory Program.\

In [36]:
print(df1[df1["description"].apply(lambda x : False if "€" not in x or "$" not in x else True)]["description"][25])

Hi there 👋

As Factorial continues on our mission of boosting the next generation of people-first SMB's, we're hiring a mid level Data Analyst Engineer to help.

The Team and you

Factorial's mission is to empower businesses to make better decisions, and data is a key component in doing so.

You’ll be part of two teams:
• the Data team as they are your peers,
• The Core domain: you will bring your data engineering & analysis expertise to that product vertical in particular, specialising in those business questions.

Data engineers are responsible of unlocking communication between the different areas despite the technological stack by offering a self-service platform that allows fast data-driven products. We act under the data mesh principle and we're not afraid to build whatever is necessary to fulfil our partner team's needs. Our data platform is already offering the following services:
• Ingestion (SaaS APIs, DDBBs, Streams)
• Storage (Data Lake, Stream Storage)
• Catalogue... (Atla